# Homework 3

Importo los modulos necesarios

In [9]:
import requests
import pandas as pd
import minsearch

from tqdm.auto import tqdm

### Importo los documentos

In [14]:
url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

In [15]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

### Importo los datos de referencia

In [16]:
ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [17]:
ground_truth[:3]

[{'question': 'When does the course begin?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'How can I get the course schedule?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'What is the link for course registration?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'}]

In [33]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
            
    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': f"{hit_rate(relevance_total):.4f}",
        'mrr': f"{mrr(relevance_total):.4f}",
    }

## Q1. Minsearch text

### Indexo los documentos en minsearch

In [22]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

### Función de busqueda con minsearch

In [23]:
def minsearch_search(query, course):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [34]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': '0.8487', 'mrr': '0.7288'}

## Q2. Vector search for question

## Q2. Cosine similarity with another vector

`np.linalg.norm(q)`: calcula la norma euclidiana (también llamada norma L2) del vector q.

In [6]:
q = result[0]
np.linalg.norm(q)

np.float64(1.0)

Si el resultado es `1`, eso implica que el vector `q` es un vector normalizado, es decir, tiene una longitud (o magnitud) de exactamente 1.

Esto es muy común en el contexto de búsqueda semántica o embeddings, donde los modelos (como `FastEmbed`, `SentenceTransformers`, `OpenAI embeddings`, etc.) normalizan automáticamente sus vectores de salida. Esto se hace para que la similitud coseno entre vectores pueda calcularse simplemente con el producto punto.

In [7]:
q.dot(q)

np.float64(1.0000000000000002)

In [8]:
doc = 'Can I still join the course after the start date?'
result_2 = embed_data(doc)

In [9]:
r = result_2[0]
r.min()

np.float64(-0.12396320482168117)

In [10]:
q.dot(r)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
len(documents)

5

In [13]:
list_docs = [ doc['text'] for doc in documents]
len(list_docs)

5

In [14]:
result_3 = embed_data(list_docs)
type(result_3)

list

In [15]:
matriz_v = np.array(result_3)

In [16]:
matriz_v.shape

(5, 512)

In [17]:
dot_product = matriz_v.dot(q)
dot_product.max()

np.float64(0.8182378150042889)

In [18]:
[(index, value) for index, value in enumerate(dot_product)]

[(0, np.float64(0.7629684696540238)),
 (1, np.float64(0.8182378150042889)),
 (2, np.float64(0.8085397398734189)),
 (3, np.float64(0.7133079015686243)),
 (4, np.float64(0.73044992343336))]

In [19]:
[(index, value) for index, value in enumerate(dot_product) if value == dot_product.max()]

[(1, np.float64(0.8182378150042889))]

## Q4. Ranking by cosine, version two

In [20]:
list_docs_v2 = [f"{doc['question']} {doc['text']}" for doc in documents]
len(list_docs_v2)

5

In [21]:
result_4 = embed_data(list_docs_v2)
type(result_4)

list

In [22]:
matriz_v2 = np.array(result_4)

In [23]:
matriz_v2.shape

(5, 512)

In [24]:
dot_product_2 = matriz_v2.dot(q)
dot_product_2.max()

np.float64(0.8514543236908068)

In [25]:
[(index, value) for index, value in enumerate(dot_product_2)]

[(0, np.float64(0.8514543236908068)),
 (1, np.float64(0.843659415911307)),
 (2, np.float64(0.8408287048502558)),
 (3, np.float64(0.7755157969663907)),
 (4, np.float64(0.8086007795043937))]

In [26]:
[(index, value) for index, value in enumerate(dot_product_2) if value == dot_product_2.max()]

[(0, np.float64(0.8514543236908068))]

### Q3 vs Q4
Mejora de Similitud al Usar `question + text` en Embeddings

#### ¿Qué se hizo?

Se modificó la forma de generar el contenido sobre el cual se aplican los embeddings. En lugar de usar únicamente el campo `text` (respuesta), se concatenaron la `question` (pregunta) y el `text` en un solo string por cada documento.

Esto generó una nueva lista de textos enriquecidos que combina pregunta y respuesta en una sola secuencia.

In [27]:
f'Q3 max:{dot_product.max()} VS Q4 max:{dot_product_2.max()} is equal = {dot_product.max()==dot_product_2.max()}'

'Q3 max:0.8182378150042889 VS Q4 max:0.8514543236908068 is equal = False'

#### Comparación de Resultados

Se compararon los valores de similitud coseno obtenidos en dos escenarios:

| Método                      | Campo utilizado              | Máxima similitud obtenida |
|----------------------------|------------------------------|----------------------------|
| Solo respuesta (`text`)    | `text`                       | 0.8182                     |
| Pregunta + respuesta (`v2`) | `question + text`            | 0.8514                     |

> Las similitudes están expresadas como `np.float64`.

#### ¿Por qué mejora el resultado?

Al usar ambos campos (`question` y `text`), se enriquece el contexto semántico del contenido. Esto permite que el modelo de embeddings:
- Capte mejor la intención del documento.
- Genere representaciones vectoriales más informativas.
- Aumente la similitud con una pregunta de consulta al incluir también el tipo de pregunta esperada.

El resultado es una mayor similitud coseno y, por ende, una mejor recuperación de información.

Este cambio simple puede tener un impacto considerable en la calidad de las respuestas recuperadas.


## Q5. Selecting the embedding model

In [28]:
list_models = [{'model': doc['model'], 'dim': doc['dim']} for doc in TextEmbedding.list_supported_models()]
list_models[:2]

[{'model': 'BAAI/bge-base-en', 'dim': 768},
 {'model': 'BAAI/bge-base-en-v1.5', 'dim': 768}]

In [29]:
list_models_sorted = sorted(list_models, key=lambda x: x['dim'])
list_models_sorted[:10]

[{'model': 'BAAI/bge-small-en', 'dim': 384},
 {'model': 'BAAI/bge-small-en-v1.5', 'dim': 384},
 {'model': 'snowflake/snowflake-arctic-embed-xs', 'dim': 384},
 {'model': 'snowflake/snowflake-arctic-embed-s', 'dim': 384},
 {'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384},
 {'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
  'dim': 384},
 {'model': 'BAAI/bge-small-zh-v1.5', 'dim': 512},
 {'model': 'Qdrant/clip-ViT-B-32-text', 'dim': 512},
 {'model': 'jinaai/jina-embeddings-v2-small-en', 'dim': 512},
 {'model': 'BAAI/bge-base-en', 'dim': 768}]

## Q6. Indexing with qdrant

### Paso 0: Ejecutar instancia de Qdrant en un contenedor Docker

Descargar la imagen y lanzar el contenedor usando los siguientes comandos:

```bash
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

In [30]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [31]:
client = QdrantClient("http://localhost", port=6333)

In [32]:
COLLECTION_NAME = "zoomcamp-llm-hw2"
MODEL_SELECT = 'BAAI/bge-small-en'
SIZE_MODEL_SELECT = client.get_embedding_size(MODEL_SELECT)
SIZE_MODEL_SELECT

384

In [33]:
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)
    
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=SIZE_MODEL_SELECT, 
        distance=models.Distance.COSINE
    ),
    timeout=60
)

print(f"Colección '{COLLECTION_NAME}' creada exitosamente.")

Colección 'zoomcamp-llm-hw2' creada exitosamente.


In [34]:
points = []

for id, doc in enumerate(documents, 1):
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=f"{doc['question']} {doc['text']}", model=MODEL_SELECT),
            payload={
                "question": doc['question'],
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)

In [35]:
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points,
    wait=True
)

print(f"{len(points)} puntos insertados en la colección usando 'upsert'.")

375 puntos insertados en la colección usando 'upsert'.


In [36]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=models.Document(
            text=query,
            model=MODEL_SELECT 
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [37]:
q = 'I just discovered the course. Can I join now?'
result = search(q)

In [38]:
result

QueryResponse(points=[ScoredPoint(id=15, version=0, score=0.8703172, payload={'question': 'The course has already started. Can I still join it?', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [39]:
result.points[0].score

0.8703172

In [40]:
result.points[0].payload['text']

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [41]:
result.points[0].payload['question']

'The course has already started. Can I still join it?'